In [1]:
!pip install emoji

In [2]:
import emoji

In [4]:
#emoji.EMOJI_UNICODE

In [7]:
emoji_dict = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [10]:
for e in emoji_dict.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


### Processing Custom Dataset

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [134]:
train_data = pd.read_csv('train_emoji.csv',header=None)
test_data = pd.read_csv('test_emoji.csv',header=None)

In [16]:
train_data.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [72]:
train = train_data.values
test = test_data.values
print(train.shape,test.shape)

(132, 4) (56, 2)


In [18]:
print(type(train))

<class 'numpy.ndarray'>


In [135]:
x_train = train_data[0]
y_train = train_data[1]

x_test = test_data[0]
y_test = test_data[1]

In [136]:
for i in range(5):
    print(x_train[i],y_train[i],emoji.emojize(emoji_dict[str(y_train[i])]))

never talk to me again 3 😓
I am proud of your achievements 2 😁
It is the worst day in my life 3 😓
Miss you so much 0 ❤️
food is life 4 🍴


### Converting sentences into Embeddings

In [56]:
f = open('glove.6B.50d.txt',encoding='utf=8')

In [57]:
embeddings_index = {}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float')
    embeddings_index[word] = coefs

In [58]:
f.close()

In [60]:
emb_dim = embeddings_index['eat'].shape
print(emb_dim)

(50,)


In [137]:
def embedding_output(x):
    maxLen = 10
    embedding_out = np.zeros((x.shape[0],maxLen,emb_dim[0]))
    for ix in range(x.shape[0]):
        x[ix] = x[ix].split()
        for ij in range(len(x[ix])):
            embedding_out[ix][ij] = embeddings_index[x[ix][ij].lower()]
                
    return embedding_out

In [138]:
train_embeddings = embedding_output(x_train)
test_embeddings = embedding_output(x_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [139]:
print(x_train[0])

['never', 'talk', 'to', 'me', 'again']


In [141]:
print(train_embeddings.shape)

(132, 10, 50)


In [99]:
print(train_embeddings.shape)
print(test_embeddings.shape)

(132, 10, 50)
(56, 10, 50)


In [105]:
from keras.utils import to_categorical

In [145]:
y_train = to_categorical(y_train,num_classes=5)
y_test = to_categorical(y_test,num_classes=5)

In [146]:
print(y_train.shape)

(132, 5)


### Defining the RNN/LSTM Model

In [102]:
from keras.models import Sequential
from keras.layers import *

In [219]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 10, 64)            29440     
_________________________________________________________________
dropout_24 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_25 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 325       
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [220]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [221]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

#checkpoint = ModelCheckpoint("bmodel.h5",monitor='val_loss',verbose=True,save_best_only=True)
#early_stop = EarlyStopping(monitor='val_acc',patience=10)
hist = model.fit(train_embeddings,y_train,validation_split=0.1,shuffle=True,epochs=100,batch_size=64)

Train on 118 samples, validate on 14 samples
Epoch 1/100
118/118 [==============================] - 6s 48ms/step - loss: 1.6195 - acc: 0.1949 - val_loss: 1.5815 - val_acc: 0.3571
Epoch 2/100
118/118 [==============================] - 0s 744us/step - loss: 1.5744 - acc: 0.2966 - val_loss: 1.5771 - val_acc: 0.2857
Epoch 3/100
118/118 [==============================] - 0s 688us/step - loss: 1.5420 - acc: 0.3475 - val_loss: 1.5761 - val_acc: 0.2857
Epoch 4/100
118/118 [==============================] - 0s 641us/step - loss: 1.5166 - acc: 0.3475 - val_loss: 1.5801 - val_acc: 0.3571
Epoch 5/100
118/118 [==============================] - 0s 687us/step - loss: 1.4929 - acc: 0.3136 - val_loss: 1.5780 - val_acc: 0.3571
Epoch 6/100
118/118 [==============================] - 0s 603us/step - loss: 1.4758 - acc: 0.3814 - val_loss: 1.5713 - val_acc: 0.2857
Epoch 7/100
118/118 [==============================] - 0s 585us/step - loss: 1.4579 - acc: 0.3644 - val_loss: 1.5508 - val_acc: 0.3571
Epoch 8/100

Epoch 61/100
118/118 [==============================] - 0s 607us/step - loss: 0.1263 - acc: 0.9492 - val_loss: 1.7759 - val_acc: 0.4286
Epoch 62/100
118/118 [==============================] - 0s 572us/step - loss: 0.0653 - acc: 0.9746 - val_loss: 1.4984 - val_acc: 0.5714
Epoch 63/100
118/118 [==============================] - 0s 621us/step - loss: 0.0738 - acc: 0.9831 - val_loss: 1.3697 - val_acc: 0.6429
Epoch 64/100
118/118 [==============================] - 0s 593us/step - loss: 0.0914 - acc: 0.9576 - val_loss: 1.0205 - val_acc: 0.5714
Epoch 65/100
118/118 [==============================] - 0s 572us/step - loss: 0.0847 - acc: 0.9746 - val_loss: 1.0089 - val_acc: 0.6429
Epoch 66/100
118/118 [==============================] - 0s 595us/step - loss: 0.0562 - acc: 0.9831 - val_loss: 1.1395 - val_acc: 0.7143
Epoch 67/100
118/118 [==============================] - 0s 594us/step - loss: 0.0567 - acc: 0.9915 - val_loss: 1.6833 - val_acc: 0.5714
Epoch 68/100
118/118 [==========================

In [222]:
train_embeddings.shape

(132, 10, 50)

In [223]:
pred = model.predict_classes(test_embeddings)

In [224]:
print(pred)

[4 3 3 2 2 2 3 0 4 2 1 2 0 3 1 3 2 2 3 2 3 0 4 3 3 3 2 0 1 2 0 1 3 2 0 1 2
 3 4 2 1 0 0 1 2 2 2 2 3 3 3 0 3 2 3 0]


In [227]:
for i in range(20):
    print(' '.join(x_test[i]))
    print(emoji.emojize(emoji_dict[str(pred[i])]))

I want to eat
🍴
he did not answer
😓
he got a raise
😓
she got me a present
😁
ha ha ha it was so funny
😁
he is a good friend
😁
I am upset
😓
We had such a lovely dinner tonight
❤️
where is the food
🍴
Stop making this joke ha ha ha
😁
where is the ball
⚾
work is hard
😁
This girl is messing with me
❤️
are you serious ha ha
😓
Let us go play baseball
⚾
This stupid grader is not working
😓
work is horrible
😁
Congratulation for having a baby
😁
stop messing around
😓
any suggestions for dinner
😁


In [226]:
model.evaluate(test_embeddings,y_test)

56/56 [==============================] - 0s 350us/step


[2.1635926280702864, 0.6249999914850507]